# 13.3 案例实战 - 新闻聚类分群模型

# 13.3.2 文本数据的读取与处理

# 1.读取数据

In [1]:
import pandas as pd
df = pd.read_excel('新闻.xlsx')
df.head()

,关键词,标题,网址,来源,时间
0,华能信托,信托公司2019年上半年经营业绩概览,http://www.financialnews.com.cn/jrsb_m/xt/zx/2...,中国金融新闻网,2019年07月23日 00:00
1,华能信托,首单信托型企业ABS获批,http://www.jjckb.cn/2018-10/23/c_137552198.htm,经济参考网,2018年10月23日 12:21
2,华能信托,华能贵诚信托孙磊:金融科技助力打造开放信托生态,https://baijiahao.baidu.com/s?id=1639276579449...,同花顺财经,2019年07月17日 10:49
3,华能信托,华能贵诚信托孙磊:金融科技已经成为信托行业重要的基础设施,https://finance.qq.com/a/20190716/007898.htm,腾讯财经,2019年07月16日 18:53
4,华能信托,格力电器股权转让意向方闭门开会 华能信托赫然在列,https://finance.sina.com.cn/trust/roll/2019-05...,新浪,2019年05月22日 22:53


In [2]:
# df.shape

# 2.中文分词

（1）简单演示

In [3]:
# 如果没有安装jieba库，可以将下面代码取消注释后运行，即可安装jieba库
# !pip install jieba

In [4]:
# 中文分词演示
import jieba
word = jieba.cut('我爱北京天安门')
for i in word:
    print(i)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/k0/hss_fhdn63l84p71mf5swlc80000gn/T/jieba.cache
Loading model cost 0.819 seconds.
Prefix dict has been built successfully.


我
爱
北京
天安门


In [5]:
# 第一条新闻标题
df.iloc[0]['标题']

'信托公司2019年上半年经营业绩概览'

In [6]:
# 第一条新闻标题中文分词
import jieba
word = jieba.cut(df.iloc[0]['标题'])
result = ' '.join(word)
print(result)

信托公司 2019 年 上半年 经营 业绩 概览


（2）实战应用

In [7]:
# 通过for循环遍历来进行所有标题的分词
import jieba
words = []
for i, row in df.iterrows():
    word = jieba.cut(row['标题'])
    result = ' '.join(word) 
    words.append(result)

In [8]:
words[0:3]  # 展示前三条新闻的分词结果

['信托公司 2019 年 上半年 经营 业绩 概览',
 '首单 信托 型 企业 ABS 获批',
 '华能 贵 诚信 托孙磊 : 金融 科技 助力 打造 开放 信托 生态']

In [9]:
# 熟悉了上面的过程后，可以把代码合并写成如下形式
import jieba
words = []
for i, row in df.iterrows():
    words.append(' '.join(jieba.cut(row['标题'])))

In [10]:
words[0:3]  # 同样展示前三条新闻的分词结果

['信托公司 2019 年 上半年 经营 业绩 概览',
 '首单 信托 型 企业 ABS 获批',
 '华能 贵 诚信 托孙磊 : 金融 科技 助力 打造 开放 信托 生态']

（3）补充知识点：遍历DataFrame表格的函数 - iterrows()函数

In [11]:
for i, row in df.iterrows():
    print(i)
    print(row)

0
关键词                                                 华能信托
标题                                    信托公司2019年上半年经营业绩概览
网址     http://www.financialnews.com.cn/jrsb_m/xt/zx/2...
来源                                               中国金融新闻网
时间                                     2019年07月23日 00:00
Name: 0, dtype: object
1
关键词                                              华能信托
标题                                       首单信托型企业ABS获批
网址     http://www.jjckb.cn/2018-10/23/c_137552198.htm
来源                                              经济参考网
时间                                  2018年10月23日 12:21
Name: 1, dtype: object
2
关键词                                                 华能信托
标题                               华能贵诚信托孙磊:金融科技助力打造开放信托生态
网址     https://baijiahao.baidu.com/s?id=1639276579449...
来源                                                 同花顺财经
时间                                     2019年07月17日 10:49
Name: 2, dtype: object
3
关键词                                            华能信托
标题                     华能贵诚信托孙磊

# 3.文字转数字：通过建立词频矩阵构造特征变量

### 3.1 文本向量化基础：建立词频矩阵

In [12]:
# CountVectorizer()函数简单演示
from sklearn.feature_extraction.text import CountVectorizer
test = ['金融 科技 厉害', '华能 信托 厉害']
vect = CountVectorizer()
X = vect.fit_transform(test)
X = X.toarray()

In [13]:
print(X)

[[0 0 1 1 1]
 [1 1 1 0 0]]


In [14]:
# 查看词袋和对应的顺序
words_bag = vect.vocabulary_
print(words_bag)

{'金融': 4, '科技': 3, '厉害': 2, '华能': 1, '信托': 0}


### 3.2 将之前所有的新闻标题进行文本向量化，从而构造特征变量

In [15]:
# 将之前所有的新闻标题进行文本向量化
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X = vect.fit_transform(words)
X = X.toarray()

In [16]:
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [17]:
# 查看所有新闻标题的词袋
words_bag = vect.vocabulary_
print(words_bag)

{'信托公司': 630, '2019': 21, '上半年': 296, '经营': 2659, '业绩': 345, '概览': 2130, '首单': 3337, '信托': 628, '企业': 538, 'abs': 103, '获批': 2827, '华能': 896, '诚信': 2947, '托孙磊': 1721, '金融': 3199, '科技': 2541, '助力': 854, '打造': 1720, '开放': 1553, '生态': 2408, '已经': 1467, '成为': 1673, '行业': 2860, '重要': 3191, '基础设施': 1135, '格力电器': 2116, '股权': 2754, '转让': 3063, '意向': 1664, '闭门': 3229, '开会': 1535, '赫然': 3013, '直击': 2465, '投资者': 1744, '见面会': 2886, '参会者': 934, '背后': 2758, '国务院': 1091, '国资委': 1098, '召开': 972, '自称': 2784, '人士': 492, '到场': 818, '2018': 20, '信托业': 629, '人均': 491, '创利': 798, '304': 44, '万元': 273, '华润': 895, '贵诚': 2992, '跌出': 3039, '万亿': 272, '俱乐部': 633, '去年': 932, '托跌出': 1724, '200': 17, '收购': 1854, '50': 60, '亿爆雷': 508, '烫手山芋': 2323, '如何': 1269, '处置': 1152, '外界': 1159, '传言': 564, '泰禾': 2221, '引进': 1566, '战投': 1695, '目标': 2464, '之一': 407, '行业动态': 2861, '山东': 1438, '年报': 1497, '规模': 2890, '缩水': 2686, '曲线': 2021, '上市': 297, '浪潮': 2241, '再度': 739, '托换帅': 1722, '孙磊': 1303, '出任': 764, '总经理': 1641, '换帅': 179

In [18]:
# 查看词袋中词的数目
len(words_bag)

3402

### 3.3 简单示例 - 取前两条新闻演示

In [19]:
# 查看前两条分词完的新闻
print(words[0])
print(words[1])

信托公司 2019 年 上半年 经营 业绩 概览
首单 信托 型 企业 ABS 获批


In [20]:
# 文本向量化
vect = CountVectorizer()  # 引入CountVectorizer()函数
X_test = vect.fit_transform(words[0:2])  # 将前两条新闻文本向量化
X_test = X_test.toarray()  # 将X_test转为数组
print(X_test)  # 查看生成的二维数组

[[1 0 1 1 0 0 1 1 1 0 0]
 [0 1 0 0 1 1 0 0 0 1 1]]


In [21]:
# 查看词袋的第一种方式
words_bag = vect.vocabulary_  # 第一种查看词袋的方式
print(words_bag)  # 查看词袋

{'信托公司': 6, '2019': 0, '上半年': 2, '经营': 8, '业绩': 3, '概览': 7, '首单': 10, '信托': 5, '企业': 4, 'abs': 1, '获批': 9}


In [22]:
# 查看词袋的第二种方式
words_bag2 = vect.get_feature_names()  # 第二种查看词袋的方法
print(words_bag2)  # 第二种查看词袋的方式

['2019', 'abs', '上半年', '业绩', '企业', '信托', '信托公司', '概览', '经营', '获批', '首单']


In [23]:
import pandas as pd
df = pd.DataFrame(X_test, columns=words_bag2)
df

,2019,abs,上半年,业绩,企业,信托,信托公司,概览,经营,获批,首单
0,1,0,1,1,0,0,1,1,1,0,0
1,0,1,0,0,1,1,0,0,0,1,1


### 3.4 将所有新闻文本向量化并展示

In [24]:
# 将之前所有的新闻标题进行文本向量化并通过pandas展示
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# 文本向量化
vect = CountVectorizer()
X = vect.fit_transform(words)  # 将分词后的内容文本向量化
X = X.toarray()

# 查看文本向量化的结果
words_bag2 = vect.get_feature_names()  # 第二种查看词袋的方法
df = pd.DataFrame(X, columns=words_bag2)
df.head()

,00700,03,04,08s,09,10,100,11,12,150,...,黄萍,黄金,黑客,黑灰产,黑金,黑马,鼓手,鼻祖,齐聚,龙风
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# # 如果想显示pandas中DataFrmae所有行，或者所有列，可以采用下面的代码
# import pandas as pd
# pd.set_option('display.max_columns', None)  # 显示所有列，如果把None改成100则显示100列
# pd.set_option('display.max_rows', None)  # 显示所有行，如果把None改成100则显示100行
# df = pd.DataFrame(X, columns=words_bag2)
# df.head()

# 13.3.3 模型搭建与使用

# 1.通过KMeans算法进行聚类分群

In [26]:
from sklearn.cluster import KMeans
kms = KMeans(n_clusters=10, random_state=123)
k_data = kms.fit_predict(df)

In [27]:
print(k_data)

[7 8 2 2 2 2 2 8 8 2 2 2 2 8 2 7 2 2 2 2 2 8 2 2 8 8 2 8 2 8 2 2 2 7 2 3 8
 2 6 2 2 2 7 2 3 8 2 6 2 2 2 7 2 3 8 2 6 2 2 2 7 2 3 8 2 6 2 2 2 7 2 3 8 2
 6 2 2 2 7 2 3 8 2 6 2 2 2 7 2 3 8 2 6 8 8 8 8 8 8 5 8 8 8 8 7 8 7 8 8 8 8
 8 8 8 8 8 8 8 7 8 7 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 7 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 7 7 8 7 7 8 8 8 8 8 7 8 8 8 8 8 8 7 8 8 8 8 8 8 8 7
 7 8 8 8 8 7 7 7 8 8 8 8 7 8 8 8 8 7 8 8 7 8 8 8 7 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 7 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 7 8 8 8 7 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 7 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 7 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 7 1 1 1 1 1 1 1 8 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 8 8 8 8 8 7 8 8 8 8 8 8 8 8 8 7 8
 8 8 8 8 8 8 8 8 8 8 8 8 

In [28]:
import numpy as np
words_ary = np.array(words)
print(words_ary[k_data == 1])  # 可以把数字1改成其他数字看看效果

['数据 科学 哪家 强 ? Python 和 R 的 对决   -   博客园 新闻 手机 版'
 '数字化 阅读 空间   |   怎么 用 Python 迅速 获取 网站 数据 ?'
 '今天 破解 了 压缩文件 的 密码 : 使用 python 轻松 编写 破解 程序'
 '程序员 如何 利用   Python   解决 女朋友 不看 天气 的 坏习惯 ?'
 '向 Excel 说 再见 , 神级 编辑器 统一 表格 与 Python'
 '谷歌 大脑 实习生 做出 Python 排版 工具 , 在线 运行 出 结果'
 '懂 Excel 就 能 轻松 入门 Python 数据分析 包 pandas ( 三 ) : 制作 成绩 条'
 '还 在 花钱 建网站 吗 ? 教你用 Python 把 github 变身 个人 网站'
 '现在 的 孩子 们 都 在 学 编程 , 是 学 C语言 还是 Python ?'
 '吴恩达 的 TensorFlow 实践 课 上线 , 有 Python 基础 就 能 听 , 4 个 月 学完'
 '让 Python 提速 超过 30 倍 的 必杀技 : Cython' '三体 究竟 有 多 可怕 ? 用 Python 建模 来 深度 了解'
 '再见 Excel ! 我 开源 了 一款 与 Python 深度 集成 的 神器 级 IDE'
 'PyTorch - Transformers : 最 先进 的 自然语言 处理 库 ( 附带 python 代码 )'
 '今日 芯声 | 要 重构 ? Python 之父 考虑 重构 Python 解释器' 'Python 之父 考虑 重构 Python 解释器'
 'Python 之父 考虑 重构 Python 解释器' 'vn . py   2.0 . 6   发布 , Python   量化 交易 框架'
 '微软 官方 上线   Python   教程 《 Develop   with   Python   on   Windows 》'
 '微软 上线 官方   Python   教程' 'Python 传奇 : 30 年 崛起 之 路'
 '如何 在 Windows 上 做 Python 开发 ? 微软 出 了 官方 教程'
 'Python   为何 能 坐 稳   A

# 2.通过DBSCAN算法进行聚类分群

In [29]:
from sklearn.cluster import DBSCAN
dbs = DBSCAN(eps=1, min_samples=3)
d_data = dbs.fit_predict(df)

In [30]:
#  DBSCAN?  # 通过在其后面加一个?，可以查看官方说明

In [31]:
print(d_data)

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1  0  1  2  3  4  5  6  7  8  0  1  2  3  4  5  6  7  8
  0  1  2  3  4  5  6  7  8  0  1  2  3  4  5  6  7  8  0  1  2  3  4  5
  6  7  8  0  1  2  3  4  5  6  7  8  0  1  2  3  4  5  6  7  8 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1

# 13.3.4 模型优化（利用余弦相似度进行优化）

1.余弦相似度基本概念

In [32]:
words_test = ['想去 华能 信托', '华能 信托 很好 想去', '华能 信托 很好 想去 华能 信托 很好 想去']

# 文本向量化
vect = CountVectorizer()
X_test = vect.fit_transform(words_test)  # 将分词后的内容文本向量化
X_test = X_test.toarray()

# 查看文本向量化的结果
words_bag2 = vect.get_feature_names()  # 第二种查看词袋的方法
df_test = pd.DataFrame(X_test, columns=words_bag2)
df_test.head()

,信托,华能,很好,想去
0,1,1,0,1
1,1,1,1,1
2,2,2,2,2


In [33]:
# 补充知识点：通过numpy库计算欧式距离
import numpy as np
dist = np.linalg.norm(df_test.iloc[0] - df_test.iloc[1])
dist

1.0

In [34]:
# 计算余弦相似度
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities  = cosine_similarity(df_test)
cosine_similarities

array([[1.       , 0.8660254, 0.8660254],
       [0.8660254, 1.       , 1.       ],
       [0.8660254, 1.       , 1.       ]])

2.余弦相似度实战 - 模型优化

In [35]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities  = cosine_similarity(df)
print(cosine_similarities)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.14142136 ... 0.         0.         0.        ]
 [0.         0.14142136 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.18490007 0.10050378]
 [0.         0.         0.         ... 0.18490007 1.         0.0836242 ]
 [0.         0.         0.         ... 0.10050378 0.0836242  1.        ]]


In [36]:
from sklearn.cluster import KMeans
kms = KMeans(n_clusters=10, random_state=123)
k_data = kms.fit_predict(cosine_similarities)

In [37]:
print(k_data)

[1 3 3 3 3 3 3 1 1 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 1 1 3 1 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 1 8 8 8 8 1 8 8 8 8 8 8 8 8 8 8 8 8 8 1 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 1 8 8 1 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 1 8 8 8 1 8 8 8 8 8 8 8 8 1 8 8 8 8 8 8 8 8 8 8 8 8 1 8 8 8 8 7 1 1
 7 7 1 7 7 7 1 1 1 7 1 7 7 7 1 7 7 7 7 1 7 7 7 7 7 1 7 1 7 1 1 7 1 1 7 7 7
 7 7 1 7 1 1 1 1 7 7 7 7 1 7 7 7 1 7 7 7 7 7 7 7 7 7 7 7 7 7 1 7 7 7 7 1 1
 7 7 1 7 7 7 7 7 7 7 7 7 1 7 7 7 7 7 7 7 7 1 7 4 4 4 4 4 4 4 1 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 1 1 1 1 1 8 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [38]:
k_data == 3

array([False,  True,  True,  True,  True,  True,  True, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True, False, False,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [39]:
# 把之前的分词结果转为数组
print(words[0:3])
words_ary = np.array(words)
print(words_ary[0:3])

['信托公司 2019 年 上半年 经营 业绩 概览', '首单 信托 型 企业 ABS 获批', '华能 贵 诚信 托孙磊 : 金融 科技 助力 打造 开放 信托 生态']
['信托公司 2019 年 上半年 经营 业绩 概览' '首单 信托 型 企业 ABS 获批'
 '华能 贵 诚信 托孙磊 : 金融 科技 助力 打造 开放 信托 生态']


In [40]:
# 查看分类结果
import numpy as np
words_ary = np.array(words)
print(words_ary[k_data == 3])  # 可以把数字3改成其他数字看看效果

['首单 信托 型 企业 ABS 获批' '华能 贵 诚信 托孙磊 : 金融 科技 助力 打造 开放 信托 生态'
 '华能 贵 诚信 托孙磊 : 金融 科技 已经 成为 信托 行业 重要 的 基础设施'
 '格力电器 股权 转让 意向 方 闭门 开会   华能 信托 赫然 在 列'
 '直击 格力电器 意向 投资者 见面会 : 参会者 华能 信托 背后 现 国务院 国资委 ...'
 '格力电器 股权 转让 意向 投资者 见面会 召开   自称 华能 信托 的 人士 到场'
 '... 或 200 亿 收购 中 江 信托   50 亿爆雷 “ 烫手山芋 ” 如何 处置 ? ; 华能 信托 ...'
 '华能 信托 是 外界 传言 泰禾 引进 战投 的 目标 之一'
 '... 或 200 亿 收购 中 江 信托   50 亿爆雷 “ 烫手山芋 ” 如何 处置 ? ; 华能 信托 ...'
 '【 行业动态 】 山东 信托 2018 年报   规模 缩水 ; 信托 曲线 上市 浪潮 再度 涌 ...'
 '华能 贵 诚信 托换帅 , 孙磊 出任 总经理' '华能 信托 换帅 总经理 孙磊 任职 资格 获批'
 '2019 年 到期 债务 达 590 亿 ! 泰禾 集团 找 华能 信托 来 救命'
 '华能 国际 : 华能 资本 是 华能 贵 诚信 托 有限公司 大 股东'
 '华能 信托 、 东莞 信托 昨日 双双 增资   今年 信托公司 注册资本 增加 总额 逾 ...'
 '华能 信托 注册 资本金 由 42 亿元 增资 至 61.94 亿'
 '时隔 近两年 再次 增资   华能 信托 注册资本 增至 61.95 亿元'
 '肖钢 密集 调研 资产 证券化 业务   走 访华 能 信托 和 中信 信托'
 '速睹 62 家 信托 上半年 业绩 ! 平安 中信 华能 位列 前 三' '华能 信托 总经理 金志 培 : 五大 秘诀 塑造 核心 竞争力'
 '华能 信托 : 信托公司 参与 消费 金融 的 新 机会 与 模式分析' '华能 信托' '新任 总经理 到位   五矿 信托 谋 突围'
 '五矿 信托 首任 总经理 辞职   接任 者 或 为 华能 信托 王卓'
 '华夏 幸福 关于 拟 与 华能 信托 签署 《 增资 协议 》 的 公告'
 '普邦 股